# STEP B

In [1]:
import cv2
from matplotlib import pyplot as plt
import numpy as np

def read_image_and_compute_keypoints(object_imgs, scene_imgs, detector):
    imgs_dict = {}

    for name_img in object_imgs:
        img_load = cv2.imread('models/' + name_img + '.jpg', cv2.IMREAD_GRAYSCALE)
        
        """
        # Enlever le centre de l'image
        row = int(img_load.shape[0]/2)
        col = int(img_load.shape[1]/2)
        pour = 0.5
        new_color = 255 # White
        img_load[int(row - row*pour) : int(row + row*pour), int(col - col*pour): int(col + col*pour)] = new_color
        #
        """

        kp, des = detector.detectAndCompute(img_load, None)
        imgs_dict[name_img] = {'kp': kp, 'des': des, 'shape': img_load.shape}

    for name_img in scene_imgs:
        img_load = cv2.imread('scenes/' + name_img + '.png', cv2.IMREAD_GRAYSCALE)
        kp, des = detector.detectAndCompute(img_load, None)
        imgs_dict[name_img] = {'kp': kp, 'des': des, 'shape': img_load.shape}

    return imgs_dict

def matching_and_localize_objects_in_scene(imgs_dict, name_scene_img, ratio_test=0.45):
    all_scene_corners = {}
    all_good ={}
    for name in object_imgs:
        object_des, object_kp = imgs_dict[name]['des'], imgs_dict[name]['kp']
        scene_des, scene_kp = imgs_dict[name_scene_img]['des'], imgs_dict[name_scene_img]['kp']
        
        #-- matching
        matches = matcher.knnMatch(object_des, scene_des, k=2)
    
        #-- store all the good matches as per Lowe's ratio test.
        good = []
        for m,n in matches:
            if m.distance < ratio_test*n.distance:
                good.append(m)
                
        if len(good) > 30:        
            #-- Get the keypoints from the good matches
            object_good_kp = np.float32([ object_kp[m.queryIdx].pt for m in good ])
            scene_good_kp = np.float32([ scene_kp[m.trainIdx].pt for m in good ])   


            #-- Get the corners of object
            h,w = imgs_dict[name]['shape'][0:2]
            object_corners = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)

            #-- Get the homography of object/scene
            H, _ =  cv2.findHomography(object_good_kp, scene_good_kp, cv2.RANSAC, 5.0)

            #-- Get the corners of object in scene
            scene_corners = cv2.perspectiveTransform(object_corners, H)


            all_scene_corners[name] = scene_corners
            all_good[name] = good
        
    return all_scene_corners, all_good

def check_best_color_matches(all_scene_corners, name_scene_img):
    
    scene_img_bgr = cv2.imread('scenes/' + name_scene_img + '.png')
    best_all_scene_corners = {}
    
    for name in all_scene_corners.keys():

        x_min = max(int(np.min(all_scene_corners[name], axis=0)[0][0]), 0)
        y_min = max(int(np.min(all_scene_corners[name], axis=0)[0][1]), 0)
        x_max = int(np.max(all_scene_corners[name], axis=0)[0][0])
        y_max = int(np.max(all_scene_corners[name], axis=0)[0][1])
        
        object_img_bgr = cv2.imread('models/' + name + '.jpg')
        
        object_img_mean_color = object_img_bgr.mean(axis=0).mean(axis=0)
        object_in_scene_img = scene_img_bgr[y_min:y_max,x_min:x_max]
        object_in_scene_img_mean_color = object_in_scene_img.mean(axis=0).mean(axis=0)   
        color_diff = np.sqrt(np.sum([value ** 2 for value in abs(object_img_mean_color - object_in_scene_img_mean_color)]))
        
        #color_diff = (np.sum([value **2 for value in (object_img_mean_color - object_in_scene_img_mean_color)]))/4
        
        if color_diff < 60:
            best_all_scene_corners[name] = all_scene_corners[name]
        
    return best_all_scene_corners

def drawn_object_lines_in_scene(img_scene, best_all_scene_corners):
    for name in best_all_scene_corners.keys():
        scene_corners = best_all_scene_corners[name]
        img_scene = cv2.polylines(img_scene, [np.int32(scene_corners)], 
                                  isClosed=True, color=(0,255,0), thickness=5)
    return img_scene

def print_object_found(best_all_scene_corners):
    for name in best_all_scene_corners.keys():  
        x_min = max(int(np.min(best_all_scene_corners[name], axis=0)[0][0]), 0)
        y_min = max(int(np.min(best_all_scene_corners[name], axis=0)[0][1]), 0)
        x_max = int(np.max(best_all_scene_corners[name], axis=0)[0][0])
        y_max = int(np.max(best_all_scene_corners[name], axis=0)[0][1])
        
        print('Product {} - {} instance/s found:'.format(name, 1))
        print('\tInstance {} position: {}, width: {}px, height: {}px'.format(1, (x_min, y_min), (x_max-x_min), (y_max-y_min)))
    
    print('_' * 80 + '\n')

def stepA():
    
    detector = cv2.SIFT_create()
    FLANN_INDEX_KDTREE = 1

        # Defining parameters for algorithm 
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)

        # Defining search params.
        # checks=50 specifies the number of times the trees in the index should be recursively traversed.
        # Higher values gives better precision, but also takes more time
    search_params = dict(checks = 50)

        # Initializing matcher
    matcher = cv2.FlannBasedMatcher(index_params, search_params)
    #matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_FLANNBASED)
    
    imgs_dict = read_image_and_compute_keypoints(object_imgs, scene_imgs, detector)
    
    
    for scene_name in scene_imgs:
        
        all_scene_corners, all_good= matching_and_localize_objects_in_scene(imgs_dict, scene_name)
        
        best_all_scene_corners = check_best_color_matches(all_scene_corners , scene_name)
        
        
        img_scene_rgb = cv2.cvtColor(cv2.imread('scenes/' + scene_name + '.png'), cv2.COLOR_BGR2RGB)
        img_scene_rgb = drawn_object_lines_in_scene(img_scene_rgb, best_all_scene_corners)

        
        print('Scene {} :'.format(scene_name))
        plt.imshow(img_scene_rgb)
        plt.show()
        print_object_found(best_all_scene_corners)

In [2]:
#-- bank of image step B
object_imgs = ['0', '1', '11', '19', '24', '25', '26']
#object_imgs = ['0', '11']
scene_imgs = ['m1', 'm2', 'm3', 'm4', 'm5']

detector = cv2.SIFT_create()
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)
matcher = cv2.FlannBasedMatcher(index_params, search_params)

In [3]:
imgs_dict = read_image_and_compute_keypoints(object_imgs, scene_imgs, detector)

In [4]:
imgs_dict

{'0': {'kp': [<KeyPoint 0x7f9214a13810>,
   <KeyPoint 0x7f9214a92f30>,
   <KeyPoint 0x7f9215411e40>,
   <KeyPoint 0x7f9214a8c270>,
   <KeyPoint 0x7f9214a8c390>,
   <KeyPoint 0x7f9214a8c300>,
   <KeyPoint 0x7f9214b1e570>,
   <KeyPoint 0x7f9214b1e0f0>,
   <KeyPoint 0x7f9214b1e480>,
   <KeyPoint 0x7f9214b1e9c0>,
   <KeyPoint 0x7f9214b1e120>,
   <KeyPoint 0x7f9214b1e150>,
   <KeyPoint 0x7f9214b1e720>,
   <KeyPoint 0x7f9214b1e540>,
   <KeyPoint 0x7f9214b1e6f0>,
   <KeyPoint 0x7f9214b1e690>,
   <KeyPoint 0x7f9214b1e660>,
   <KeyPoint 0x7f9214b1e630>,
   <KeyPoint 0x7f9214b1e750>,
   <KeyPoint 0x7f9214b1e600>,
   <KeyPoint 0x7f9214b1e5d0>,
   <KeyPoint 0x7f9214b1e510>,
   <KeyPoint 0x7f9214b1e840>,
   <KeyPoint 0x7f9214b1e7e0>,
   <KeyPoint 0x7f9214b1e810>,
   <KeyPoint 0x7f9214b1e090>,
   <KeyPoint 0x7f9214b1e780>,
   <KeyPoint 0x7f9214b1e3c0>,
   <KeyPoint 0x7f9214b1e900>,
   <KeyPoint 0x7f9214b1e8a0>,
   <KeyPoint 0x7f9214b1e9f0>,
   <KeyPoint 0x7f9214b1e8d0>,
   <KeyPoint 0x7f9214b1e2d0>,

In [5]:
def matching_and_store_good_matches(imgs_dict, name_scene_img, ratio_test = 0.5):
    all_good_matches = {}
    all_good_keypoints = {}
    for name in object_imgs:
        object_des, object_kp = imgs_dict[name]['des'], imgs_dict[name]['kp']
        scene_des, scene_kp = imgs_dict[name_scene_img]['des'], imgs_dict[name_scene_img]['kp']

        #-- matching
        matches = matcher.knnMatch(object_des, scene_des, k=2)

        #-- store all the good matches as per Lowe's ratio test.

        good = []
        for m,n in matches:
            if m.distance < 0.5*n.distance:
                good.append(m)

        good_kp = {}
        for m in good:
            good_kp[scene_kp[m.trainIdx]] = object_kp[m.queryIdx]

    
    #-- put in the dict of matches of all images if there are more matches than the threshold
    
        if len(good) > 30:
            all_good_matches[name] = good
            all_good_keypoints[name] = good_kp
    
    return all_good_matches, all_good_keypoints

In [6]:
all_good_matches, all_good_keypoints = matching_and_store_good_matches(imgs_dict, 'm1')
all_good_keypoints

{'0': {<KeyPoint 0x7f91ef29f600>: <KeyPoint 0x7f92149e0b70>,
  <KeyPoint 0x7f91ef29fbd0>: <KeyPoint 0x7f9214afd6c0>,
  <KeyPoint 0x7f91ef2a0450>: <KeyPoint 0x7f9214b02090>,
  <KeyPoint 0x7f91ef2a0690>: <KeyPoint 0x7f9214b04780>,
  <KeyPoint 0x7f91ef2a08d0>: <KeyPoint 0x7f9214b087b0>,
  <KeyPoint 0x7f91ef2a09c0>: <KeyPoint 0x7f9214b061b0>,
  <KeyPoint 0x7f91ef2a0d20>: <KeyPoint 0x7f9214b0c2a0>,
  <KeyPoint 0x7f91ef2a1120>: <KeyPoint 0x7f921603cb40>,
  <KeyPoint 0x7f91ef2a04b0>: <KeyPoint 0x7f921603ce70>,
  <KeyPoint 0x7f91ef2a1330>: <KeyPoint 0x7f921603fc30>,
  <KeyPoint 0x7f91ef2a0d50>: <KeyPoint 0x7f9216041bd0>,
  <KeyPoint 0x7f91ef2a2b40>: <KeyPoint 0x7f9216041c30>,
  <KeyPoint 0x7f91ef2a2b70>: <KeyPoint 0x7f9216041c60>,
  <KeyPoint 0x7f91ef2a1720>: <KeyPoint 0x7f9216039090>,
  <KeyPoint 0x7f91ef2a1660>: <KeyPoint 0x7f92160391e0>,
  <KeyPoint 0x7f91ef2a1a20>: <KeyPoint 0x7f9216038570>,
  <KeyPoint 0x7f91ef2a3a50>: <KeyPoint 0x7f9216049ed0>,
  <KeyPoint 0x7f91ef2a4450>: <KeyPoint 0x7f

In [7]:
def compute_barycenter_of_found_object_keypoints(all_good_keypoints):
    all_barycenter = {}
    for name in all_good_keypoints.keys():
        good_kp = all_good_keypoints[name]
        object_xc = np.mean([ kp.pt[0] for kp in good_kp.values() ], dtype='float')
        object_yc = np.mean([ kp.pt[1] for kp in good_kp.values() ], dtype='float')
        
        all_barycenter[name] = (object_xc, object_yc)
        
    return all_barycenter

In [8]:
all_barycenter = compute_barycenter_of_found_object_keypoints(all_good_keypoints)
all_barycenter

{'0': (549.498402615811, 821.3395929539457),
 '24': (142.96356725168752, 248.03028450431404),
 '25': (301.6193597521101, 257.0644633974348),
 '26': (141.86475553571807, 245.28104630464352)}

In [9]:
def compute_GHT_for_all_matches(all_good_keypoints, all_barycenter):
    
    all_hough_space = {}
    for name in all_good_keypoints.keys():
    
        hough_space = {}
        good_kp = all_good_keypoints[name]
        object_xc, object_yc = all_barycenter[name]

        for scene_kp, object_kp in good_kp.items():


            entry = {}

            v = ((object_xc - object_kp.pt[0]), (object_yc - object_kp.pt[1]))
            ratio_of_size = scene_kp.size / object_kp.size
            relative_angle = scene_kp.angle - object_kp.angle
            x_c = scene_kp.pt[0] + ratio_of_size * (np.cos(relative_angle) * v[0] - np.sin(relative_angle) * v[1])
            y_c = scene_kp.pt[1] + ratio_of_size * (np.sin(relative_angle) * v[0] + np.cos(relative_angle) * v[1])

            entry['x_c'] = x_c
            entry['y_c'] = y_c
            entry['ratio_of_size'] = ratio_of_size
            entry['relative_angle'] = relative_angle

            hough_space[(object_kp, scene_kp)] = entry
        
        all_hough_space[name] = hough_space
        
    return all_hough_space

In [24]:
all_hough_space = compute_GHT_for_all_matches(all_good_keypoints, all_barycenter)
all_hough_space

{'0': {(<KeyPoint 0x7f92149e0b70>,
   <KeyPoint 0x7f91ef29f600>): {'x_c': 869.7717408473618, 'y_c': 136.43710349130316, 'ratio_of_size': 0.20894468466288907, 'relative_angle': -0.628265380859375},
  (<KeyPoint 0x7f9214afd6c0>,
   <KeyPoint 0x7f91ef29fbd0>): {'x_c': 765.6589305647713, 'y_c': 301.52418620856014, 'ratio_of_size': 0.21370777567898583, 'relative_angle': -3.834930419921875},
  (<KeyPoint 0x7f9214b02090>,
   <KeyPoint 0x7f91ef2a0450>): {'x_c': 869.1119873256517, 'y_c': 391.47838527786223, 'ratio_of_size': 0.2172305697350106, 'relative_angle': 2.04876708984375},
  (<KeyPoint 0x7f9214b04780>,
   <KeyPoint 0x7f91ef2a0690>): {'x_c': 768.0869956886227, 'y_c': 354.74370652032184, 'ratio_of_size': 0.22918312328026508, 'relative_angle': 2.8911285400390625},
  (<KeyPoint 0x7f9214b087b0>,
   <KeyPoint 0x7f91ef2a08d0>): {'x_c': 869.4114770219168, 'y_c': 146.81520153856445, 'ratio_of_size': 0.22919689096798046, 'relative_angle': -0.62744140625},
  (<KeyPoint 0x7f9214b061b0>,
   <KeyPoint

In [11]:
def compute_bins_for_one_scene(all_hough_space, imgs_dict, img_scene_name):

    all_values_of_bins = {}
    
    for name in all_hough_space.keys():

        list_ratio_of_size = [ entry['ratio_of_size'] for entry in all_hough_space[name].values() ]
        ratio_of_size_global = np.median(list_ratio_of_size)

        list_relative_angle = [entry['relative_angle'] for entry in all_hough_space[name].values()]


        BIN_PRECISION_FACTOR = 0.25
        ANGLE_BINS = 7
        ANGLE_BIN_SIZE_COEFF = 0.1
        SCALE_BIN_SIZE_COEFF = 0.1

        object_shape = imgs_dict[name]['shape']
        scene_shape = imgs_dict[img_scene_name]['shape']

        x_bin_size = ratio_of_size_global * object_shape[1] * BIN_PRECISION_FACTOR
        y_bin_size = ratio_of_size_global * object_shape[0] * BIN_PRECISION_FACTOR
        x_bins = int(np.ceil(scene_shape[1] / x_bin_size) + 2)
        y_bins = int(np.ceil(scene_shape[0] / y_bin_size) + 2)
        x_min = scene_shape[1] / 2 - x_bins / 2 * x_bin_size
        y_min = scene_shape[0] / 2 - y_bins / 2 * y_bin_size

        angle_bin_size = np.std(list_relative_angle) * ANGLE_BIN_SIZE_COEFF
        angle_bin_center = np.mean(list_relative_angle)
        angle_min = angle_bin_center - ANGLE_BINS / 2 * angle_bin_size
        angle_max = angle_bin_center + ANGLE_BINS / 2 * angle_bin_size

        scale_bin_size = np.std(list_ratio_of_size) * SCALE_BIN_SIZE_COEFF
        scale_bin_center = np.mean(list_ratio_of_size)
        scale_min = 0 
        scale_max = scale_bin_center * 2 
        scale_bins = int((scale_max - scale_min) / scale_bin_size)

        bins_values = {}
        bins_values['x_bins'] = x_bins
        bins_values['y_bins'] = y_bins
        bins_values['x_min'] = x_min
        bins_values['y_min'] = y_min
        bins_values['x_bin_size'] = x_bin_size
        bins_values['y_bin_size'] = y_bin_size
        bins_values['scale_bins'] = scale_bins
        bins_values['scale_min'] = scale_min
        bins_values['scale_bin_size'] = scale_bin_size
        bins_values['angle_min'] = angle_min
        bins_values['angle_bin_size'] = angle_bin_size  

        all_values_of_bins[name] = bins_values
    
    return all_values_of_bins

In [13]:
all_values_of_bins = compute_bins_for_one_scene(all_hough_space, imgs_dict, 'm1')
all_values_of_bins

{'0': {'x_bins': 25,
  'y_bins': 8,
  'x_min': -79.44551412380758,
  'y_min': -110.29533645197068,
  'x_bin_size': 63.315641129904606,
  'y_bin_size': 85.94883411299267,
  'scale_bins': 20,
  'scale_min': 0,
  'scale_bin_size': 0.03215630504264163,
  'angle_min': -21.34274293227928,
  'angle_bin_size': 7.3309366093378205},
 '24': {'x_bins': 19,
  'y_bins': 6,
  'x_min': -105.94759560452678,
  'y_min': -136.72891169468062,
  'x_bin_size': 86.09974690573966,
  'y_bin_size': 123.4096372315602,
  'scale_bins': 209,
  'scale_min': 0,
  'scale_bin_size': 0.012327700422739053,
  'angle_min': -14.775403093433473,
  'angle_bin_size': 3.8042562468066397},
 '25': {'x_bins': 20,
  'y_bins': 7,
  'x_min': -103.95173802020281,
  'y_min': -149.5173452588952,
  'x_bin_size': 81.59517380202028,
  'y_bin_size': 109.4335272168272,
  'scale_bins': 139,
  'scale_min': 0,
  'scale_bin_size': 0.008014336299391136,
  'angle_min': -11.984462096105329,
  'angle_bin_size': 3.0330346900862164},
 '26': {'x_bins': 

In [16]:
def voting(all_values_of_bins, all_hough_space):
    
    ANGLE_BINS = 7
    
    all_votes = {}
    all_accumulator = {}
    
    for name in all_hough_space.keys():
        
        bins_values = all_values_of_bins[name]
        hough_space = all_hough_space[name]
        
        accumulator = np.zeros((bins_values['x_bins'], bins_values['y_bins'], ANGLE_BINS, bins_values['scale_bins']))

        votes = {}

        for kps, v in hough_space.items():
            try:
                for x in range(0, 2):
                    for y in range(0, 2):
                        for z in range(0, 2):
                            for w in range(0, 2):                
                                i = int(np.floor((v['x_c'] - bins_values['x_min'] + (x - 1 / 2) * bins_values['x_bin_size']) / bins_values['x_bin_size']))
                                j = int(np.floor((v['y_c'] - bins_values['y_min'] + (y - 1 / 2) * bins_values['y_bin_size']) / bins_values['y_bin_size']))
                                k = int(np.floor((v['relative_angle'] - bins_values['angle_min'] + (z - 1 / 2) * bins_values['angle_bin_size']) / bins_values['angle_bin_size']))
                                l = int(np.floor((v['ratio_of_size'] - bins_values['scale_min'] + (w - 1 / 2) * bins_values['scale_bin_size']) / bins_values['scale_bin_size']))
                                if i >= 0 and j >= 0 and k >= 0 and l >= 0:
                                    accumulator[i, j, k, l] += 1
                                    votes[(i, j, k, l)] = votes.get((i, j, k, l), [])
                                    votes[(i, j, k, l)].append(kps)
            except: pass
            
        all_votes[name] = votes
        all_accumulator[name] = accumulator
        
    return all_votes, all_accumulator

In [33]:
all_votes, all_accumulator = voting(all_values_of_bins, all_hough_space)
all_votes

{'0': {(14, 2, 2, 5): [(<KeyPoint 0x7f92149e0b70>, <KeyPoint 0x7f91ef29f600>)],
  (14, 2, 2, 6): [(<KeyPoint 0x7f92149e0b70>, <KeyPoint 0x7f91ef29f600>),
   (<KeyPoint 0x7f9214b087b0>, <KeyPoint 0x7f91ef2a08d0>)],
  (14, 2, 3, 5): [(<KeyPoint 0x7f92149e0b70>, <KeyPoint 0x7f91ef29f600>)],
  (14, 2, 3, 6): [(<KeyPoint 0x7f92149e0b70>, <KeyPoint 0x7f91ef29f600>),
   (<KeyPoint 0x7f9214b087b0>, <KeyPoint 0x7f91ef2a08d0>),
   (<KeyPoint 0x7f9214b0c2a0>, <KeyPoint 0x7f91ef2a0d20>)],
  (14, 3, 2, 5): [(<KeyPoint 0x7f92149e0b70>, <KeyPoint 0x7f91ef29f600>),
   (<KeyPoint 0x7f9216039090>, <KeyPoint 0x7f91ef2a1720>)],
  (14, 3, 2, 6): [(<KeyPoint 0x7f92149e0b70>, <KeyPoint 0x7f91ef29f600>),
   (<KeyPoint 0x7f9214b087b0>, <KeyPoint 0x7f91ef2a08d0>),
   (<KeyPoint 0x7f9216041c30>, <KeyPoint 0x7f91ef2a2b40>),
   (<KeyPoint 0x7f9216041c60>, <KeyPoint 0x7f91ef2a2b70>),
   (<KeyPoint 0x7f9216039090>, <KeyPoint 0x7f91ef2a1720>)],
  (14, 3, 3, 5): [(<KeyPoint 0x7f92149e0b70>, <KeyPoint 0x7f91ef29f600>),

In [34]:
import skimage.morphology as sk

In [53]:
def compute_correspondences(all_votes, all_accumulator):
    
    all_global_correspondences = {}
    
    for name in all_votes.keys():
    
        mask = sk.local_maxima(all_accumulator[name])
        all_accumulator[name][mask != 1] = 0

        T_Q = 10
        global_correspondences = []

        for b in list(np.argwhere(all_accumulator[name] >= T_Q)): # thresholding the accumulator to come up with few maxima 
            global_correspondences.append([all_accumulator[name][tuple(b)], all_votes[name][tuple(b)]])
            
        all_global_correspondences[name] = global_correspondences
        
    return all_global_correspondences
    

In [54]:
all_global_correspondences = compute_correspondences(all_votes, all_accumulator)
all_global_correspondences['0'][0][1]

[(<KeyPoint 0x7f9214b061b0>, <KeyPoint 0x7f91ef2a09c0>),
 (<KeyPoint 0x7f92160391e0>, <KeyPoint 0x7f91ef2a1660>),
 (<KeyPoint 0x7f9216049ed0>, <KeyPoint 0x7f91ef2a3a50>),
 (<KeyPoint 0x7f921604cd80>, <KeyPoint 0x7f91ef2a4450>),
 (<KeyPoint 0x7f921604cdb0>, <KeyPoint 0x7f91ef2a4480>),
 (<KeyPoint 0x7f921604f540>, <KeyPoint 0x7f91ef2a49f0>),
 (<KeyPoint 0x7f92160562d0>, <KeyPoint 0x7f91ef2a54e0>),
 (<KeyPoint 0x7f9203636f30>, <KeyPoint 0x7f91ef2a5870>),
 (<KeyPoint 0x7f9203638ea0>, <KeyPoint 0x7f91ef2a5a50>),
 (<KeyPoint 0x7f9203638ed0>, <KeyPoint 0x7f91ef2a5a80>),
 (<KeyPoint 0x7f920363ec90>, <KeyPoint 0x7f91ef2a62a0>),
 (<KeyPoint 0x7f920363fb70>, <KeyPoint 0x7f91ef2a6390>),
 (<KeyPoint 0x7f920363fc30>, <KeyPoint 0x7f91ef2a6300>),
 (<KeyPoint 0x7f9203642ea0>, <KeyPoint 0x7f91ef2a66f0>),
 (<KeyPoint 0x7f9203642ed0>, <KeyPoint 0x7f91ef2a6720>),
 (<KeyPoint 0x7f9203644ae0>, <KeyPoint 0x7f91ef2a69f0>),
 (<KeyPoint 0x7f9203648540>, <KeyPoint 0x7f91ef2a6e40>),
 (<KeyPoint 0x7f920364a8d0>, <K

In [55]:
def compute_objects_corners_in_one_scene(all_global_correspondences, scene_name):
    
    all_scene_corners = []
    
    for name in all_global_correspondences.keys():
        h,w = imgs_dict[name]['shape'][0:2]
        object_corners = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)

        for i in range(len(all_global_correspondences[name])):
            scene_corners = {}
            keypoints = all_global_correspondences[name][i]
            object_pts = np.float32([kp[1].pt for kp in keypoints[1]]).reshape(-1, 1, 2)
            scene_pts = np.float32([kp[0].pt for kp in keypoints[1]]).reshape(-1, 1, 2)
            H, _ =  cv2.findHomography(object_pts, scene_pts, cv2.RANSAC, 5.0)


            scene_corners[name] = cv2.perspectiveTransform(object_corners, H)
            
            all_scene_corners.append(scene_corners)
    
    return all_scene_corners

In [64]:
all_scene_corners = compute_objects_corners_in_one_scene(all_global_correspondences, 'm1')
all_scene_corners

[{'0': array([[[-2.3669963e+03,  2.6366647e+02]],
  
         [[-1.2815109e+04,  1.8812291e+04]],
  
         [[-1.5392002e+04, -1.5728827e+05]],
  
         [[ 1.1657218e+03,  5.1049976e+01]]], dtype=float32)},
 {'0': array([[[-2.3669963e+03,  2.6366647e+02]],
  
         [[-1.2815109e+04,  1.8812291e+04]],
  
         [[-1.5392002e+04, -1.5728827e+05]],
  
         [[ 1.1657218e+03,  5.1049976e+01]]], dtype=float32)},
 {'25': array([[[-1894.3633  ,    32.13788 ]],
  
         [[-1799.72    ,  1314.1199  ]],
  
         [[ -856.61707 ,  1351.3885  ]],
  
         [[ -924.68726 ,     8.313677]]], dtype=float32)},
 {'25': array([[[-1206.7273 ,   117.33696]],
  
         [[-3951.355  ,  2367.0542 ]],
  
         [[-3069.191  ,  3307.7517 ]],
  
         [[ -603.0638 ,    81.78471]]], dtype=float32)}]

In [66]:
def drawn_object_lines_in_scene(img_scene, all_scene_corners):
    for dic in all_scene_corners:
        for values in dic.values():
            scene_corners = values
            img_scene = cv2.polylines(img_scene, [np.int32(scene_corners)], 
                                  isClosed=True, color=(0,255,0), thickness=5)
    return img_scene

In [58]:
img_scene = cv2.imread('scenes/m1.png')
img_scene = drawn_object_lines_in_scene(img_scene, all_scene_corners)
plt.imshow(img_scene)

NameError: name 'scene_corners' is not defined

In [ ]:
# function that split an image into (n_bins_heigth x n_bins_width) bins and returns a dictonary
# that contains the means of the 3 color channels of the (n_bins_heigth x n_bins_width) bins
def split_image_into_N_x_M_bins_with_intensity_means(image, n_bins_width = 19, n_bins_heigth = 6):
    img_N_x_M_bins = {}
    
    img2 = np.copy(image)
    
    img_width = image.shape[1]
    img_height = image.shape[0]
    
    step_width = int(img_width / n_bins_width)
    step_height = int(img_height / n_bins_heigth)
    
    r = 0
    c = 0
    
    #print('img_height: ', img_height)
    #print('step_height: ', step_height)
    if img_height != 0 and step_height != 0 and img_width != 0 and step_width != 0:
        for row in np.arange(0, img_height, step_height):
            c = 0
            cv2.line(img2,(0, row),(img_width, row),(0,0,0),3) 
            for col in np.arange(0, img_width, step_width):
                # print('row {} col {}'.format(r,c))
                if row + 2 * step_height > img_height and col + 2 * step_width > img_width:
                    partial_r_channel, partial_g_channel, partial_b_channel = cv2.split(image[row:, col:])

                elif row + 2 * step_height > img_height:
                    partial_r_channel, partial_g_channel, partial_b_channel = cv2.split(image[row:, col : col + step_width])

                elif col + 2 * step_width > img_width:
                    partial_r_channel, partial_g_channel, partial_b_channel = cv2.split(image[row : row + step_height, col :])
                else:
                    partial_r_channel, partial_g_channel, partial_b_channel = cv2.split(
                        image[row : row + step_height, col : col + step_width])

                cv2.line(img2,(col, 0),(col, img_height),(0,0,0),3) 
                if r < n_bins_heigth and c < n_bins_width:
                    # save means of the 3 channels (r,g,b) of each bin
                    img_N_x_M_bins[r, c] = (np.mean(partial_r_channel), np.mean(partial_g_channel), np.mean(partial_b_channel))
                    # IF YOU WANT TO VISUALIZE EACH BIN UNCOMMENT THE FOLLOWING LINES:
                    # plot each bin in blue channel color
                    plt.imshow(cv2.cvtColor(partial_b_channel, cv2.COLOR_BGR2RGB))
                    plt.show()


                c += 1
            r += 1
        #plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
        #plt.show()  
    return img_N_x_M_bins

In [ ]:
img = cv2.imread('scenes/e1.png')
split_image_into_N_x_M_bins_with_intensity_means(img)

In [ ]:
bins_values